# IBM Data Science with Python Capstone - Week 3 Assignment

### Contains a code to create and clean a DataFrame with location data from Toronto. Later, some attempts for clustering and map creation follow.


In [5]:
# Code to parse Wikipedia page and extract data into DataFrame

!pip install folium

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
#import geocoder
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import sklearn
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import json

print('Installed all')

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,

In [6]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(wiki_url,'lxml')
print(soup.prettify())

wiki_table = soup.find_all('table',{'class':'wikitable sortable'})
table = wiki_table[0]

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":867606113,"wgRevisionId":867606113,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [7]:
header_row = table.find('tr')
col_names = []

# Check if columns are available
for header in header_row.find_all('th'):

    name = ' '.join(header.find_all(text=True))
    name.replace('\n', '')
    col_names.append(name)

In [8]:
content = []
temp_post =[]
temp_boro = []
temp_neighbor = []

for tr in table.find_all('tr'):

    temp_list =[]

    td_all = tr.find_all('td')

    if not td_all:
        continue

    post, boro, neighbor = [td.text.strip() for td in td_all[0:3]]

    temp_list.append(post)
    temp_list.append(boro)
    temp_list.append(neighbor)

    content.append(temp_list)

In [9]:
df_toronto = pd.DataFrame(data=content, columns=['PostalCode', 'Borough', 'Neighborhood'])

df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']

df_group = df_toronto.groupby(['PostalCode','Borough'])['Neighborhood'].agg(lambda x: ', '.join(x)).reset_index()

print(df_group.shape)

(103, 3)


In [ ]:
# Geocoder not working, therefore only as comment:

#for postal_code in df_group['PostalCode']:

    #lat_long_coords = None

   # while(lat_long_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        #lat_long_coords = g.latlng

    #latitude = lat_long_coords[0]
    #longitude = lat_long_coords[1]

In [13]:
data_csv = 'http://cocl.us/Geospatial_data'

df_geo = pd.read_csv(data_csv)

df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

df_torontoGEO = pd.merge(df_group, df_geo,how='left', on='PostalCode')

In [15]:
# Map of Toronto:

address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [25]:

# Loop to add markers:
for lat, lng, borough, neighborhood in zip(df_torontoGEO['Latitude'], df_torontoGEO['Longitude'], df_torontoGEO['Borough'], df_torontoGEO['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)   
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
    location=[lat, lng],
    popup=label).add_to(map_toronto)

map_toronto

In [28]:
# API Foursquare and function from the labs to get the Toronto venues. Is prone to request errors, however.

CLIENT_ID = 'STUXIOFXH55TITYVUTGKIDG3CAG50XD0B3SM2XE2WWB3IU1Q'
CLIENT_SECRET = 'AU3QRD5UCY5MRYBX3K2E3MMBOGCVNTA2WMHPZTDEN4IJ4CLZ'
VERSION = '20180605'
Limit = 100


def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            Limit)

        # make the GET request

        results = requests.get(url).json()['response']['groups'][0]['items']


        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude',
                  'Neighborhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return(nearby_venues)


toronto_venues = getNearbyVenues(names=df_torontoGEO['Neighborhood'], latitudes=df_torontoGEO['Latitude'], longitudes=df_torontoGEO['Longitude'])


KeyError: 'groups'

In [27]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

NameError: name 'toronto_venues' is not defined

In [ ]:
num_clusters = 6

toronto_grouped_cluster = toronto_grouped.drop('Neighborhood', 1)

kmeans_clustering = KMeans(n_clusters=num_clusters, random_state=0).fit(toronto_grouped_cluster)

kmeans_clustering.labels_[0:10]

toronto_merged = toronto_grouped

toronto_merged['Clusters'] = kmeans_clustering.labels_

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i+x+(i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

# End